# Mine Recognition

## Step 1: Import Libraries

In [1]:
import os
import json
import pandas as pd
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

## Step 2: Load and Prepare the Data

In [2]:
# Load the JSON file
with open('images/data.json', 'r') as f:
    data = json.load(f)

# Convert to DataFrame
df = pd.DataFrame(data)
df['image_path'] = df['file_name']  # Ensure file names match your actual path

# Split data into train and validation
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

## Step 3: Create a Custom Dataset

In [ ]:
class MineDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # Load image paths and labels based on folder structure
        for label, class_name in enumerate(['false', 'true']):
            class_dir = os.path.join(img_dir, class_name)
            for img_file in os.listdir(class_dir):
                self.image_paths.append(os.path.join(class_dir, img_file))
                self.labels.append(label)  # 0 for 'false', 1 for 'true'

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define image transformations with augmentation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomRotation(10),      # Randomly rotate the image by ±10 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Randomly change brightness, contrast, saturation, and hue
    transforms.Resize((224, 224)),      # Resize to the target size
    transforms.ToTensor(),               # Convert the image to a tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize with ImageNet statistics
])

# Initialize datasets and loaders
img_dir = 'images'  # Parent directory containing 'true' and 'false' subdirectories
dataset = MineDataset(img_dir=img_dir, transform=transform)

# Optionally split into train and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

## Step 4: Define the Model
Load a pre-trained model, such as ResNet18, and modify it for binary classification.

In [4]:
class MineClassifier(nn.Module):
    def __init__(self):
        super(MineClassifier, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        
        # Pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 28 * 28, 256)  # Adjust dimensions based on input size
        self.fc2 = nn.Linear(256, 1)
        
        # Activation function and dropout
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Convolutional layers with pooling and ReLU activation
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        
        # Flatten the tensor for the fully connected layer
        x = x.view(-1, 128 * 28 * 28)  # Adjust this based on input dimensions
        
        # Fully connected layers with dropout
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

In [5]:
class EnhancedMineClassifier(nn.Module):
    def __init__(self):
        super(EnhancedMineClassifier, self).__init__()
        
        # Convolutional layers with Batch Normalization
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)

        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        
        # Pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers
        self.fc1 = nn.Linear(512 * 7 * 7, 1024)  # Adjust dimensions based on input size
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 1)
        
        # Activation function and dropout
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Convolutional layers with ReLU, BatchNorm, and Pooling
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = self.pool(self.relu(self.bn4(self.conv4(x))))
        x = self.pool(self.relu(self.bn5(self.conv5(x))))
        
        # Flatten the tensor for the fully connected layer
        x = x.view(-1, 512 * 7 * 7)  # Adjust this based on your input dimensions
        
        # Fully connected layers with dropout
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        
        return x


## Step 5: Set Loss Function and Optimizer
Binary cross-entropy loss with logits and Adam optimizer.

In [6]:
# Initialize the custom model

model = MineClassifier()
# model = EnhancedMineClassifier()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device

device(type='cuda')

## Step 6: Training Loop
Train the model and evaluate on the validation set after each epoch.

In [7]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    # Training phase
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).float()
        
        optimizer.zero_grad()
        outputs = model(images).squeeze(1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    # Validation phase
    model.eval()
    val_loss = 0.0
    all_labels = []
    all_preds = []
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device).float()
            outputs = model(images).squeeze(1)
            
            # Calculate loss
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            
            # Store predictions and true labels
            preds = torch.sigmoid(outputs) > 0.5  # Apply threshold to get binary predictions
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
    
    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, zero_division=1)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader.dataset):.4f}, "
          f"Val Loss: {val_loss/len(val_loader.dataset):.4f}, "
          f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

Epoch [1/10], Loss: 0.6901, Val Loss: 0.1022, Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1 Score: 1.0000
Epoch [2/10], Loss: 1.1643, Val Loss: 5.7736, Accuracy: 0.0000, Precision: 1.0000, Recall: 0.0000, F1 Score: 0.0000
Epoch [3/10], Loss: 2.9097, Val Loss: 0.7926, Accuracy: 0.5000, Precision: 1.0000, Recall: 0.5000, F1 Score: 0.6667
Epoch [4/10], Loss: 0.5627, Val Loss: 0.1671, Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1 Score: 1.0000
Epoch [5/10], Loss: 0.6486, Val Loss: 0.1724, Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1 Score: 1.0000
Epoch [6/10], Loss: 0.4412, Val Loss: 0.2318, Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1 Score: 1.0000
Epoch [7/10], Loss: 0.3593, Val Loss: 0.4627, Accuracy: 0.7500, Precision: 1.0000, Recall: 0.7500, F1 Score: 0.8571
Epoch [8/10], Loss: 0.3038, Val Loss: 0.1574, Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1 Score: 1.0000
Epoch [9/10], Loss: 0.2216, Val Loss: 0.3299, Accuracy: 0.7500, Precisio

## Step 7: Predict on New Images
Now, we can classify new images that don't have labels.

In [9]:
def classify_image(model, image_path):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension and move to device
    
    with torch.no_grad():
        output = model(image)
        prediction = torch.sigmoid(output).item()  # Sigmoid to get probability
        print(f"{prediction*100:.4f}%")
        return 1 if prediction > 0.5 else 0  # Threshold of 0.5 for binary classification

# Example of classifying a new image
new_image_path = 'images/unknown0.jpg'
prediction = classify_image(model, new_image_path)
print(f"Prediction: {'Mine' if prediction == 1 else 'No Mine'}")


8.7087%
Prediction: No Mine
